In [151]:
import lazypredict
import pandas as pd
from sklearn.preprocessing import RobustScaler
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, chi2, f_classif

In [166]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
df_X_last = pd.read_csv("data/X_train_features.csv", index_col=0)
df_X_richard = pd.read_csv("data/template_features.csv", index_col=0)
df_X_tim = pd.read_csv("full_waveform_features.csv", index_col=0)
df_X_pyHRV = pd.read_csv("data/pyHRV_features.csv", index_col=0)
df_X_hrv_analysis = pd.read_csv("data/hrv-analysis_features.csv")
df_X = pd.concat((df_X_tim,df_X_richard),axis=1)
df_X = pd.concat((df_X, df_X_pyHRV),axis=1)
df_X = pd.concat((df_X, df_X_hrv_analysis),axis=1)
# df_X = pd.concat((df_X, df_X_last),axis=1)
df_Y = pd.read_csv("data/Y_train.csv", index_col="id")

Remove highly correlated features

In [167]:
def rm_corr(X):
    corr_matrix = X.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Find features with correlation greater than 0.9
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    print("Removed columns: ", len(to_drop))
    # Drop features 
    X.drop(to_drop, axis=1, inplace=True)
    #X_test.drop(to_drop, axis=1, inplace=True)
    
    return X

In [168]:
df_X = rm_corr(df_X)

Removed columns:  31


Normalize

In [169]:
df_X.replace([np.inf, -np.inf], np.nan, inplace=True)
transformer = RobustScaler()
X = transformer.fit_transform(df_X)

In [170]:
X.shape

(5117, 93)

Impute missing values

In [171]:
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
X = imp_median.fit_transform(X)

Feature Selection

In [149]:
X_new = SelectKBest(f_classif, k=80).fit_transform(X, df_Y)
X_new.shape

(5117, 80)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, df_Y,stratify=df_Y,test_size=.5,random_state=0)
clf = LazyClassifier(verbose=0, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

 97%|███████████████████████████████████████████████████████████████████████████████▏  | 28/29 [00:35<00:01,  1.54s/it]

In [ ]:
# XGB, LGBM, RandomForestClassifier, ExtraTrees

In [121]:
df_X_hsv.shape

(5117, 22)